In [9]:
import numpy as np
import pandas as pd
import gc
working_dir = '~/Project/pred-future-sales'

In [2]:
train_test = pd.read_pickle(working_dir + '/data/interim/train_test.pkl')

In [5]:
train_test.head()

,date_block_num,shop_id,item_id,item_cnt_month,ID,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,...,item_cnt_month_subtype_avg_lag_1,item_cnt_month_city_avg_lag_1,item_cnt_month_shop_category_avg_lag_1,item_cnt_month_shop_type_avg_lag_1,item_cnt_month_shop_subtype_avg_lag_1,item_cnt_month_item_city_avg_lag_1,delta_item_price_lag,delta_shop_month_revenue_lag_1,item_shop_first_sale,item_first_sale
4488710,12,2,27,0.0,0.0,0,19,5,12,0.0,...,1.075009,0.148118,0.955326,0.808824,0.934844,0.0,-0.282795,-0.363988,12,12
4488711,12,2,30,0.0,0.0,0,40,11,6,0.0,...,0.291505,0.148118,0.046246,0.051719,0.046246,0.0,-0.483264,-0.363988,11,11
4488712,12,2,31,0.0,0.0,0,37,11,1,0.0,...,0.232808,0.148118,0.059459,0.051719,0.059459,0.0,-0.137618,-0.363988,11,11
4488713,12,2,32,1.0,0.0,0,40,11,6,0.0,...,0.291505,0.148118,0.046246,0.051719,0.046246,0.0,-0.407143,-0.363988,12,12
4488714,12,2,33,1.0,0.0,0,37,11,1,1.0,...,0.232808,0.148118,0.059459,0.051719,0.059459,1.0,-0.225177,-0.363988,12,12


In [6]:
train_test.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'ID',
       'city_code', 'item_category_id', 'type_code', 'subtype_code',
       'item_cnt_month_lag_1', 'item_cnt_month_lag_2', 'item_cnt_month_lag_3',
       'item_cnt_month_lag_6', 'item_cnt_month_lag_12',
       'item_cnt_month_avg_lag_1', 'item_cnt_month_item_avg_lag_1',
       'item_cnt_month_item_avg_lag_2', 'item_cnt_month_item_avg_lag_3',
       'item_cnt_month_item_avg_lag_6', 'item_cnt_month_item_avg_lag_12',
       'item_cnt_month_shop_avg_lag_1', 'item_cnt_month_shop_avg_lag_2',
       'item_cnt_month_shop_avg_lag_3', 'item_cnt_month_shop_avg_lag_6',
       'item_cnt_month_shop_avg_lag_12', 'item_cnt_month_category_avg_lag_1',
       'item_cnt_month_category_avg_lag_2',
       'item_cnt_month_category_avg_lag_3',
       'item_cnt_month_category_avg_lag_6',
       'item_cnt_month_category_avg_lag_12', 'item_cnt_month_type_avg_lag_1',
       'item_cnt_month_subtype_avg_lag_1', 'item_cnt_month_city_avg_lag_1',


In [7]:
x_train = train_test[train_test.date_block_num < 33].drop(['item_cnt_month'], axis=1)
y_train = train_test[train_test.date_block_num < 33]['item_cnt_month']
x_valid = train_test[train_test.date_block_num == 33].drop(['item_cnt_month'], axis=1)
y_valid = train_test[train_test.date_block_num == 33]['item_cnt_month']
x_test = train_test[train_test.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [10]:
del train_test


NameError: name 'train_test' is not defined

In [11]:
gc.collect();

In [14]:
import xgboost as xgb

In [26]:
params = {
    'booster': 'gbtree', 
    'objective': 'reg:linear', 
    'eta': 0.1, 
    'gamma': 0.0, 
    'alpha': 0.0, 
    'lambda': 1.0, 
    'min_child_weight': 1, 
    'max_depth': 5, 
    'subsample': 0.8, 
    'colsample_bytree': 0.8, 
    'random_state': 71, 
}

In [18]:
dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_valid, y_valid)
dtest = xgb.DMatrix(x_test)

In [19]:
num_round = 50

In [ ]:
watchlist = [(dtrain, 'train', (dvalid, 'eval'))]
model = xgb.train(params, dtrain, num_round, evals=watchlist, early_stopping_rounds=50)

[15:37:09] Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[15:37:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[0]	train-rmse:1.15898
Will train until train-rmse hasn't improved in 50 rounds.
[15:37:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[1]	train-rmse:1.11713
[15:37:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[2]	train-rmse:1.08186


KeyboardInterrupt: 

[3]	train-rmse:1.05276
[15:37:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[4]	train-rmse:1.02628
[15:37:50] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[5]	train-rmse:1.00464
[15:37:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[6]	train-rmse:0.986113
[15:38:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[7]	train-rmse:0.970184
[15:38:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[8]	train-rmse:0.956272
[15:38:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[9]	train-rmse:0.944874
[15:38:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 